God-sejong에서 한글 데이터를 난독화된 한글 데이터로 변환한다.
미리 작성된 js를 그대로 사용하기 때문에, 따로 더 개발할 필요는 없을 것으로 예상된다.
따라서 웹 크롤링을 사용해 데이터 셋을 구축할 것이고, HTML과 동적으로 상호작용해야 하므로, 라이브러리는 selenium을 사용할 것이다.
구현 시나리오는 다음과 같다.
1. WebDriver 초기화
1. 웹 페이지 열기
1. 준비한 한국어 문장 수 만큼(for), 아래 시도를 반복한다.
   1. 문장을 입력한다. (id: input-area)
   1. 변환 버튼을 클릭한다.
   1. 결과 값에 변화가 있을 때 까지 아래 시도를 반복한다.
      1. 초기 결과 값을 저장한다.
      1. 결과 값을 가져온다.
      1. 이전 결과 값과 현재 값이 다를 경우, 데이터를 저장한다.
1. WebDriver를 종료한다

In [1]:
# 한국어 데이터만 추출하기
import json

# 경로 설정
file_path = './aihub-kor-eng-translation-dataset/01.데이터/1.Training/원천데이터/일상생활및구어체_한영_train_set.json'

with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# "ko_original" 필드 추출
ko_original_list = [item["ko_original"] for item in data['data']]
eng_list = [item['en'] for item in data['data']]

In [2]:
print(ko_original_list[2])
print(eng_list[2])

>속옷을?
>Underwear?


In [ ]:
# 이전 코드. 사용하지 말 것. API로 개선함.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 1. WebDriver 초기화
driver = webdriver.Edge('./web-driver(edge)/msedgedriver.exe')

# 2. 웹 페이지 열기 (Live Server로 테스트를 진행함)
driver.get('http://127.0.0.1:5500/God-sejong/index.html')

# 저장할 데이터 셋 정리
converted_kor = []

# * 필요한 요소 미리 구하기
input_area = driver.find_element(By.ID, 'input-area')
convert_button = driver.find_element(By.ID, 'convert_button')

# 3. 준비한 한국어 문장 수 만큼 아래 시도를 반복한다.
for to_conv in ko_original_list:
    cnt = 0 
    # 문장을 입력한다.
    input_area.clear()
    input_area.send_keys(to_conv)
    
    result = driver.find_element(By.ID, 'output-area')
    previous_result_text = result.get_attribute('value')
    
    # 변환 버튼을 클릭한다.
    convert_button.click()

    # 결과 값에 변화기 있을 때 까지 아래 시도를 반복한다.
    while True:
        # 결과 값 가져오기
        result = driver.find_element(By.ID, 'output-area')
        current_result_text = result.get_attribute('value')

        # 이전 결과 값과 현재 결과 값이 다를 경우 데이터를 가져옴
        if current_result_text != previous_result_text:
            converted_kor.append(current_result_text)
            break
        
        previous_result_text = current_result_text
        
        #멈춤 현상 처리
        cnt += 1
        if cnt > 50:
            convert_button.click()
            cnt = 0

print(converted_kor[0:10])

In [2]:
import requests

converted_kor = []  # 결과 값을 저장할 리스트

url = 'http://localhost:3000/api/noise'

for text in ko_original_list:
    try:
        response = requests.post(url, data={'conv_this': text})
        if response.status_code == 200:
            converted_text = response.json()['converted_message']
            converted_kor.append(converted_text)
        else:
            converted_kor.append(None)  # 요청 실패 시 None 값으로 저장
    except requests.exceptions.RequestException as e:
        print(f'Error converting text "{text}": {str(e)}')
        converted_kor.append(None)  # 에러 발생 시 None 값으로 저장

print('Conversion completed.')
print(converted_kor[2], ko_original_list[2])

Error converting text "이것은 보안 카메라입니다.": HTTPConnectionPool(host='localhost', port=3000): Max retries exceeded with url: /api/noise (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001843ACAE890>: Failed to establish a new connection: [WinError 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다'))
Error converting text "그런데 안 먹고 나가면 혼나요, 해줬는데.": HTTPConnectionPool(host='localhost', port=3000): Max retries exceeded with url: /api/noise (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000018354514110>: Failed to establish a new connection: [WinError 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다'))
Error converting text ">으악, 으악~": HTTPConnectionPool(host='localhost', port=3000): Max retries exceeded with url: /api/noise (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001835450A7D0>: Failed to establish a new connection: [WinError 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다'))
Error converting

KeyboardInterrupt: 

In [ ]:
import json

# result 데이터 생성
result = [{'ko_original': origin, 'ko_converted': converted} for origin, converted in zip(ko_original_list, converted_kor)]

# JSON 파일로 저장
with open('ko_converted_data.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [9]:
import js2py as js

#js2py 모듈을 이용해서 변환하기
# ./noise_model/mainscript.js 파일 변환하기
js.translate_file('./(NO)_noise_model/mainscript.js', './conv_main.py')

In [13]:
# 번역 테스트
import conv_main
print(conv_main.convert_text('안녕하세요. 변환을 테스트 합니다.'))

AttributeError: module 'conv_main' has no attribute 'convert_text'

In [27]:
import js2py

code = """
function test(a, b)
{
    return a + b;
}
function test2(a, b)
{
    return a - b;
}
"""

testFunction = js2py.eval_js(code)

print(testFunction(1, 2))


-1


In [3]:
#내용 수정하기
# JSON 파일로 저장
with open('ko_converted_data.json', 'r', encoding='utf-8') as f:
    add_eng_data = json.load(f)

for i, item in enumerate(add_eng_data):
    item['en_origin'] = eng_list[i]

# 수정된 데이터 저장
with open("ko_converted_data_modified.json", "w", encoding="utf-8") as f:
    json.dump(add_eng_data, f, ensure_ascii=False, indent=4)